# Исследование надежности заемщиков

In [84]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [85]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [86]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [92]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`.**

In [95]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных встречаются артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце days_employed. Обработка значения в этом столбце: замена всех отрицательных значений положительными с помощью метода abs().

In [99]:
data['days_employed'] = data['days_employed'].abs()

**Выведение медианного значения трудового стажа `days_employed` в днях.**

In [102]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

**Выведение перечня уникальных значений столбца `children`.**

In [105]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**Удаление строк, в которых встречаются аномальные значения из датафрейма `data`.**

In [108]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

** Выведение перечня уникальных значений столбца `children` чтобы убедиться, что артефакты удалены.**

In [111]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

**Заполнение пропусков в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [115]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Выведение количества пропущенных значений для каждого столбца с помощью двух методов.**

In [118]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Замена вещественного типа данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [122]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Обработка неявных дубликатов в столбце `education`. Приведение значений к нижнему регистру.**

In [126]:
data['education'] = data['education'].str.lower()

**Выведение на экран количества строк-дубликатов в данных, удаление их.**

In [129]:
data.duplicated().sum()

71

In [131]:
data = data.drop_duplicates()

### Категоризация данных

**На основании указанных диапазонов, создание в датафрейме `data` столбца `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [135]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [137]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведение на экран перечня уникальных целей взятия кредита из столбца `purpose`.**

In [140]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создание функции, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [143]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [145]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследование данных и ответы на вопросы

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [149]:
children_pivot = data.pivot_table(index=['children'], values='debt', aggfunc={'count', 'sum'})
children_pivot = children_pivot.rename(columns={'count':'Общее','sum':'Кредиты'})
children_pivot['Доля в %'] = round(100*children_pivot['Кредиты']/children_pivot['Общее'], 1)
children_pivot = children_pivot.sort_values(by='Доля в %', ascending=True)
display(children_pivot)

,Общее,Кредиты,Доля в %
children,,,
5,9,0,0.0
0,14091,1063,7.5
3,330,27,8.2
1,4808,444,9.2
2,2052,194,9.5
4,41,4,9.8


Вывод: взаимосвязь между количеством детей и возвратом кредита в срок прослеживается. Наименьшую просрочку возврата денежных средств (или же ее отсутствие) допускают люди у которых нет детей. Можно предположить, что люди, у которых дети отсутсвуют, обладают хорошей платежеспособностью, что напрямую связано с отсутствием иждивенцев. Гипотезу о связи отсутствия детей и пониженной финансовой нагрузки подтверждает наибольший процент невозвратности кредитов у семей с одним и двумя детьми, при этом, в семьях с двумя детьми процент невозвратности кредитов больше. Данных по семьям с 3, 4, 5 детьми очень мало, в связи с чем из анализа они исключены.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [153]:
age_pivot = data.pivot_table(index=['family_status'], values='debt', aggfunc={'count','sum'})
age_pivot = age_pivot.rename(columns={'count':'Общее','sum':'Кредиты'})
age_pivot['Доля в %'] = round(100*age_pivot['Кредиты']/age_pivot['Общее'], 1)
age_pivot = age_pivot.sort_values(by='Доля в %', ascending=True)
display(age_pivot)

,Общее,Кредиты,Доля в %
family_status,,,
вдовец / вдова,951,63,6.6
в разводе,1189,84,7.1
женат / замужем,12261,927,7.6
гражданский брак,4134,385,9.3
Не женат / не замужем,2796,273,9.8


Вывод: наибольшая вероятность возврата кредита в срок у людей находящихся в разводе. Предположительно, это свзяано с тем, что большее количество людей уже в возрасте, соответственно более финансово грамотны и ответственны. Люди в браке имеют нейтральную кредитную историю, что может объясняться как наличием детей, так и проблемой с финансами у партнера что требует дополнительных расходов в связи с общим бюджетом и общей ответственностью (или наоборот партнер помогает гасить кредит). Самый низкий процент возврата кредитов у людей не состоящих в браке и сожителей (находящихся в гражданском браке), это может объясняться молодым возрастом и более низкой степенью ответственности за действия партнера, поскольку приобретаемое имущество не делится пополам, соответственно в случае банкротства сожитель не останется "ни с чем".

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [157]:
income_pivot = data.pivot_table(index=['total_income_category'], values='debt', aggfunc={'count','sum'})
income_pivot = income_pivot.rename(columns={'count':'Общий','sum':'Кредиты'})
income_pivot['Доля в %'] = round(100*income_pivot['Кредиты']/income_pivot['Общий'], 1)
income_pivot = income_pivot.sort_values(by='Доля в %', ascending=True)
display(income_pivot)

,Общий,Кредиты,Доля в %
total_income_category,,,
D,349,21,6.0
B,5014,354,7.1
A,25,2,8.0
C,15921,1353,8.5
E,22,2,9.1


Вывод: наибольший процент возврата кредитов у людей с уровнем дохода от тридцати до пятидесяти тысяч рублей, что может объясняться наличием страха не осуществить выплату в срок, что повлечет дополнительные расходы. Люди с доходом от 50 до 200 имеют высокую вероятность, относительно остальных, не вернуть кредит, что может объясняться наличием нескольких кредитов, а также иждивенцев.

#### Как разные цели кредита влияют на его возврат в срок?

In [161]:
purpose_pivot = data.pivot_table(index=['purpose_category'], values='debt', aggfunc={'count','sum'})
purpose_pivot = purpose_pivot.rename(columns={'count':'Общий','sum':'Кредиты'})
purpose_pivot['Доля в %'] = round(100*purpose_pivot['Кредиты']/purpose_pivot['Общий'], 1)
purpose_pivot = purpose_pivot.sort_values(by='Доля в %', ascending=True)
display(purpose_pivot)

,Общий,Кредиты,Доля в %
purpose_category,,,
операции с недвижимостью,10751,780,7.3
проведение свадьбы,2313,183,7.9
операции с автомобилем,4279,400,9.3
получение образования,3988,369,9.3


Вывод: здесь наблюдается корреляция. Вероятность возврата кредита заметно выше у людей, стремящихся создать семью , чем у людей, которые берут потребительский кредит на вещи существенно ниже по стоимости. Здесь, вероятно, играет роль величина суммы кредита

### Общий вывод.

Цель исследования:
Основываясь на статистике о платёжеспособности клиентов нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

Поставленные вопросы:
1 Есть ли зависимость между количеством детей и возвратом кредита в срок?
2 Есть ли зависимость между семейным положением и возвратом кредита в срок?
3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4 Как разные цели кредита влияют на его возврат в срок?

Для целей исследования были предобработаны данные:
Пропущенные в двух столбцах (days_employed и total_income) данные заполнены медианным значением по каждому типу. 
Данные с отрицательными значениями в столбцах (days_employed) заменены положительными.
Данные с аномальными значениями в столбце children удалены.
В столбце education все значения приведены к нижнему регистру, удалены дубликаты. 
Кредитополучатели разделены на пять категорий по уровню дохода: 0–30000; 30001–50000; 50001–200000; 200001–1000000;1000001 и выше.
Цели взятия кредита разделены на четыре основные категории: операции с автомобилем; операции с недвижимостью; проведение свадьбы; получение образования.

Выводы:
Взаимосвязь между количеством детей и возвратом кредита в срок прослеживается. Наименьшую просрочку возврата денежных средств допускают люди у которых нет детей, наибольший процент невозвратности кредитов у семей с одним и двумя детьми, при этом, в семьях с двумя детьми процент невозвратности кредитов больше.  
Наибольшая вероятность возврата кредита в срок у людей находящихся в разводе. Люди в браке имеют нейтральную кредитную историю, а самый низкий процент возврата кредитов у людей не состоящих в браке и сожителей.
Наибольший процент возврата кредитов у людей с уровнем дохода от тридцати до пятидесяти тысяч рублей. Люди с доходом от пятидесяти до двухста тысяч имеют высокую вероятность, относительно остальных, не вернуть кредит.
Вероятность возврата кредита заметно выше у людей, стремящихся создать семью , чем у людей, которые берут потребительский кредит на вещи существенно ниже по стоимости. 

Соответственно, семейное положение и количество детей клиента как правило влияет на факт погашения кредита в срок. Но, следует учитывать также и такие категории как уровень заработной платы, а также цели взятия кредита. 